# Q1. Running Elastic
Run Elastic Search 8.17.6, and get the cluster information. If you run it on localhost, this is how you do it:
```
curl localhost:9200
```
What's the `version.build_hash` value?

In [1]:
!curl localhost:9200

{
  "name" : "94be89505406",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "0C_JKH3xQAeZUiarpz0UkQ",
  "version" : {
    "number" : "8.17.6",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "dbcbbbd0bc4924cfeb28929dc05d82d662c527b7",
    "build_date" : "2025-04-30T14:07:12.231372970Z",
    "build_snapshot" : false,
    "lucene_version" : "9.12.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   541  100   541    0     0  20587      0 --:--:-- --:--:-- --:--:-- 20807


Answer: `dbcbbbd0bc4924cfeb28929dc05d82d662c527b7`

# Getting the data

In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

# Q2. Index the data

Which function do you use for adding your data to elastic?

Answer: `index`

In [3]:
from elasticsearch import Elasticsearch

In [4]:
es_client = Elasticsearch("http://localhost:9200")

In [6]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-homework-1"

# create index
es_client.indices.create(index=index_name, body=index_settings)

C:\Users\chetv\AppData\Local\Temp\ipykernel_41548\2384178953.py:19: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-homework-1'})

In [7]:
from tqdm import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|████████████████████████████████████████████████| 948/948 [00:54<00:00, 17.29it/s]


# Q3. Searching

Now let's search in our index.

We will execute a query "How do execute a command on a Kubernetes pod?".

Use only `question` and `text` fields and give question a boost of 4, and use "type": `"best_fields"`.

What's the score for the top ranking result?

Answer: 44.5

In [8]:
query = "How do execute a command on a Kubernetes pod?"

In [16]:
search_query = {
    "size": 5, 
    "query": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"], 
                    "type": "best_fields"
            }
        }
    }

In [17]:
response = es_client.search(index=index_name, body=search_query)

C:\Users\chetv\AppData\Local\Temp\ipykernel_41548\449040615.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=search_query)


In [22]:
response['hits']['max_score']

44.50556

# Q4. Filtering
Now ask a different question: "How do copy a file to a Docker container?".

This time we are only interested in questions from `machine-learning-zoomcamp`.

Return 3 results. What's the 3rd question returned by the search engine?


Answer: `'How do I copy files from a different folder into docker container’s working directory?'`

In [60]:
question = "How do copy a file to a Docker container?"

search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": question, 
                        "fields": ["question^4", "text"], 
                        "type": "best_fields"
                    }
                },
                "filter": { 
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

In [61]:
response = es_client.search(index=index_name, body=search_query)

C:\Users\chetv\AppData\Local\Temp\ipykernel_41548\449040615.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=search_query)


In [62]:
response['hits']['hits'][-1]['_source']['question']

'How do I copy files from a different folder into docker container’s working directory?'

# Q5. Building a prompt
Now we're ready to build a prompt to send to an LLM.

Take the records returned from Elasticsearch in Q4 and use this template to build the context. 
```
context_template = """
    Q: {question}
    A: {text}
""".strip()
```

Separate context entries by two linebreaks (\n\n)

In [63]:
# get documents from search results
search_results_docs = []

for hit in response['hits']['hits']:
    search_results_docs.append(hit['_source'])

In [64]:
# template for context
context_template = """
Q: {question}
A: {text}
""".strip()

In [65]:
# function to create a prompt from search results docs
def build_context(search_results: list) -> str:
    
    context_elements = []
    
    for doc in search_results:
        context = context_template.format(question=doc['question'], text=doc['text'])
        context_elements.append(context)
    
    return "\n\n".join(context_elements)

In [66]:
context = build_context(search_results_docs)
print(context)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

Now use the context you just created along with the "How do I execute a command in a running docker container?" question to construct a prompt using the template below:
```
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```
What's the length of the resulting prompt? (use the len function)

In [67]:
question = "How do I execute a command in a running docker container?"

In [68]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [69]:
prompt = prompt_template.format(question=question, context=context)

In [70]:
len(prompt)

1462

# Q6. Tokens
When we use the OpenAI Platform, we're charged by the number of tokens we send in our prompt and receive in the response.

The OpenAI python package uses tiktoken for tokenization:
```
pip install tiktoken
```
Let's calculate the number of tokens in our query:

In [72]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

In [73]:
tokens = encoding.encode(prompt)
print(len(tokens))

322


# Bonus: generating the answer (ungraded)
Let's send the prompt to OpenAI. What's the response?

In [75]:
import openai
from openai import OpenAI
import os

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
client = OpenAI(api_key=OPENAI_API_KEY)

In [77]:
response = client.chat.completions.create(
                model='gpt-4o',
                messages=[{"role": "user", "content": prompt}]
        )

In [90]:
response.choices[0].message.content

'To execute a command in a running Docker container, you can use the `docker exec` command. First, find the container ID by using `docker ps`. Then, execute the command in the specific container using the following syntax:\n\n```\ndocker exec -it <container-id> <command>\n```\n\nFor example, to start a bash session in the container, you would use:\n\n```\ndocker exec -it <container-id> bash\n```'

# Bonus: calculating the costs (ungraded)
Suppose that on average per request we send 150 tokens and receive back 250 tokens.

How much will it cost to run 1000 requests?

Current costs (June, 2025):
gpt-4o (per 1M tokens) 

Input: $2.50

Output: $10.00

In [108]:
prices = {"input": 2.50 / 1_000_000,
          "output": 10/ 1_000_000}

bonus_question_costs = (150 * prices['input'] + 250 * prices['output']) * 1000

print(f"Bonus question costs: ${bonus_question_costs:.4f}")

Bonus question costs: $2.8750


In [105]:
# Q6 response costs
usage = {"input": response.usage.prompt_tokens,
        "output": response.usage.completion_tokens
}

total_costs = sum(prices[k] * usage[k] for k in usage)

In [107]:
usage

{'input': 329, 'output': 90}

In [106]:
print(f"Credits used: ${total_costs:.4f}")

Credits used: $0.0017
